In [1]:
!pip install facenet_pytorch

In [2]:
!pip install matplotlib

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob
import os
# from tensorflow.keras.models import load_model

# from tensorflow.keras.models import Model
import os
import warnings


from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim



In [4]:

class CustomDataset(Dataset):
    def __init__(self, csv_file, get_pairs, num_pairs = None, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'male': 0, 'female': 1}
        self.ethnicity_mapping = {'white': 0, 'black': 1, 'asian': 2, 'hispanic': 3}
        self.data['nameNum'] = self.data['name'].astype('category').cat.codes
        self.data['nameNum'] = self.data['nameNum'].astype(int)
        self.get_pairs = get_pairs
        self.num_pairs = num_pairs


    def __len__(self):
        return len(self.data)


    
    def getAgeLabel(self,value1):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        if(class_ranges[0][0]<=value1 and value1<class_ranges[0][1]):
            return 0
        elif(class_ranges[1][0]<=value1 and value1<class_ranges[1][1]):
            return 1
        elif(class_ranges[2][0]<=value1 and value1<class_ranges[2][1]):
            return 2
        elif(class_ranges[3][0]<=value1 and value1<class_ranges[3][1]):
            return 3
        else:
            return 0
   

    def __getitem__(self, idx):
        if self.get_pairs == False:
            row = self.data.iloc[idx]
            image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename']  # Assuming images are in a folder named 'images'
            
            try:
                image = Image.open(image_path)
            except Exception as e:
                # Handle the error, for example, you can return a placeholder image
                print("here")
                #self.__getitem__(idx + 1)
                #image = Image.new('RGB', (224, 224))  # Create a blank image
            
            image = Image.open(image_path)
            age = row['age']
            
            if(row['age']<=0):
                age=35
            label = {
                'age': self.getAgeLabel(age),
                'gender': self.gender_mapping.get(row['gender'], 0),  # -1 for unknown
                'ethnicity': self.ethnicity_mapping.get(row['ethnicity'], 0),
                'age1':age,
                'name' : row['nameNum']
            
            }
            #print(row['name'])
            if self.transform:
                image = self.transform(image)

            return image, label

        # else
        else:
            if self.num_pairs == None:
                print("Num of Pairs not specified.Setting to 50.")
                self.num_pairs = 50
            row = self.data.iloc[idx]
            image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename']  # Assuming images are in a folder named 'images'
            
            try:
                image = Image.open(image_path)
            except Exception as e:
                # Handle the error, for example, you can return a placeholder image
                print("here")
                #self.__getitem__(idx + 1)
                #image = Image.new('RGB', (224, 224))  # Create a blank image
            
            image = Image.open(image_path)
            
            #print(row['name'])
            if self.transform:
                image = self.transform(image)

            genuine_images = []
            imposter_images = []
            if idx + self.num_pairs < len(self.data):
                st = max(0, idx - self.num_pairs)
                end = idx + self.num_pairs
            else:
                st = idx - self.num_pairs + 1
                end = len(self.data)
            for i in range(st, end):
                row_pair = self.data.iloc[i]
                
                if row_pair['nameNum'] == row['nameNum']:

                    image_path_pair = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/' + row_pair['name'] + '/' + row_pair['filename']

                    image_pair = Image.open(image_path_pair)
                    #print(row['name'])
                    if self.transform:
                        image_pair = self.transform(image_pair)

                    
                    genuine_images.append(image_pair)
                if len(genuine_images) == self.num_pairs:
                    break
                
            i = idx
            while True:
                i = (i + 1000) % len(self.data)
                row_pair = self.data.iloc[i]
                
                if row_pair['nameNum'] != row['nameNum']:

                    image_path_pair = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/' + row_pair['name'] + '/' + row_pair['filename']

                    image_pair = Image.open(image_path_pair)
                    #print(row['name'])
                    if self.transform:
                        image_pair = self.transform(image_pair)

                    
                    imposter_images.append(image_pair)
                if len(imposter_images) == self.num_pairs:
                    break
            # print(images)
            # print(len(genuine_images))
            # print(len(imposter_images))
            return image, genuine_images, imposter_images



In [5]:
# csv_file = '/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv'  # Replace with the actual path to your CSV file
# df = pd.read_csv(csv_file)

# # Create a list to store the indices of rows with missing files
# rows_to_remove = []
# count=0
# # Iterate through the DataFrame and check if the files exist
# for index, row in df.iterrows():
#     image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename'] 
#     if not os.path.exists(image_path):
#         rows_to_remove.append(index)
#         count=count+1
# df = df.drop(rows_to_remove)
# df.to_csv(csv_file, index=False)
# print(count)

In [6]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to the desired size
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


In [7]:
# trainSet = CustomDataset('/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/celebSetIdentity.csv', transform=transform, get_pairs=True, num_pairs=25)
# trainloader = DataLoader(trainSet, batch_size=16, shuffle=True, num_workers = 2)

# testSet = CustomDataset('/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/celebSetIdentityTest.csv', transform=transform, get_pairs=False)
# testloader = DataLoader(testSet, batch_size=16, shuffle=False)

trainSet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv', get_pairs=False, transform=transform)
trainloader = DataLoader(trainSet, batch_size=32, shuffle=True, num_workers=2)

testSet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv', get_pairs=False, transform=transform)
testloader = DataLoader(testSet, batch_size=32, shuffle=False, num_workers=2)


In [8]:
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

In [9]:
# tempIter = iter(testloader)
# images,gen_pairs, imp_pairs = next(tempIter)
# # imshow(images[0])
# # print(labels['age'][0],labels['gender'][0],labels['ethnicity'][0])
# # print(image_pairs)
# imshow(images[0])
# imshow(gen_pairs[0][0])
# imshow(imp_pairs[0][0])

In [10]:
from collections import namedtuple
import torch
import torch.nn as nn
from torch.nn import Dropout
from torch.nn import MaxPool2d
from torch.nn import Sequential
from torch.nn import Conv2d, Linear
from torch.nn import BatchNorm1d, BatchNorm2d
from torch.nn import ReLU, Sigmoid
from torch.nn import Module
from torch.nn import PReLU
import os

def build_model(model_name='ir_50'):
    if model_name == 'ir_101':
        return IR_101(input_size=(112,112))
    elif model_name == 'ir_50':
        return IR_50(input_size=(112,112))
    elif model_name == 'ir_se_50':
        return IR_SE_50(input_size=(112,112))
    elif model_name == 'ir_34':
        return IR_34(input_size=(112,112))
    elif model_name == 'ir_18':
        return IR_18(input_size=(112,112))
    else:
        raise ValueError('not a correct model name', model_name)

def initialize_weights(modules):
    """ Weight initilize, conv2d and linear is initialized with kaiming_normal
    """
    for m in modules:
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight,
                                    mode='fan_out',
                                    nonlinearity='relu')
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight,
                                    mode='fan_out',
                                    nonlinearity='relu')
            if m.bias is not None:
                m.bias.data.zero_()


class Flatten(Module):
    """ Flat tensor
    """
    def forward(self, input):
        return input.view(input.size(0), -1)


class LinearBlock(Module):
    """ Convolution block without no-linear activation layer
    """
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(LinearBlock, self).__init__()
        self.conv = Conv2d(in_c, out_c, kernel, stride, padding, groups=groups, bias=False)
        self.bn = BatchNorm2d(out_c)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class GNAP(Module):
    """ Global Norm-Aware Pooling block
    """
    def __init__(self, in_c):
        super(GNAP, self).__init__()
        self.bn1 = BatchNorm2d(in_c, affine=False)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.bn2 = BatchNorm1d(in_c, affine=False)

    def forward(self, x):
        x = self.bn1(x)
        x_norm = torch.norm(x, 2, 1, True)
        x_norm_mean = torch.mean(x_norm)
        weight = x_norm_mean / x_norm
        x = x * weight
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        feature = self.bn2(x)
        return feature


class GDC(Module):
    """ Global Depthwise Convolution block
    """
    def __init__(self, in_c, embedding_size):
        super(GDC, self).__init__()
        self.conv_6_dw = LinearBlock(in_c, in_c,
                                     groups=in_c,
                                     kernel=(7, 7),
                                     stride=(1, 1),
                                     padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(in_c, embedding_size, bias=False)
        self.bn = BatchNorm1d(embedding_size, affine=False)

    def forward(self, x):
        x = self.conv_6_dw(x)
        x = self.conv_6_flatten(x)
        x = self.linear(x)
        x = self.bn(x)
        return x


class SEModule(Module):
    """ SE block
    """
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = Conv2d(channels, channels // reduction,
                          kernel_size=1, padding=0, bias=False)

        nn.init.xavier_uniform_(self.fc1.weight.data)

        self.relu = ReLU(inplace=True)
        self.fc2 = Conv2d(channels // reduction, channels,
                          kernel_size=1, padding=0, bias=False)

        self.sigmoid = Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return module_input * x



class BasicBlockIR(Module):
    """ BasicBlock for IRNet
    """
    def __init__(self, in_channel, depth, stride):
        super(BasicBlockIR, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False),
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1, bias=False),
            BatchNorm2d(depth),
            PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1, bias=False),
            BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class BottleneckIR(Module):
    """ BasicBlock with bottleneck for IRNet
    """
    def __init__(self, in_channel, depth, stride):
        super(BottleneckIR, self).__init__()
        reduction_channel = depth // 4
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False),
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, reduction_channel, (1, 1), (1, 1), 0, bias=False),
            BatchNorm2d(reduction_channel),
            PReLU(reduction_channel),
            Conv2d(reduction_channel, reduction_channel, (3, 3), (1, 1), 1, bias=False),
            BatchNorm2d(reduction_channel),
            PReLU(reduction_channel),
            Conv2d(reduction_channel, depth, (1, 1), stride, 0, bias=False),
            BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class BasicBlockIRSE(BasicBlockIR):
    def __init__(self, in_channel, depth, stride):
        super(BasicBlockIRSE, self).__init__(in_channel, depth, stride)
        self.res_layer.add_module("se_block", SEModule(depth, 16))


class BottleneckIRSE(BottleneckIR):
    def __init__(self, in_channel, depth, stride):
        super(BottleneckIRSE, self).__init__(in_channel, depth, stride)
        self.res_layer.add_module("se_block", SEModule(depth, 16))


class Bottleneck(namedtuple('Block', ['in_channel', 'depth', 'stride'])):
    '''A named tuple describing a ResNet block.'''


def get_block(in_channel, depth, num_units, stride=2):

    return [Bottleneck(in_channel, depth, stride)] +\
           [Bottleneck(depth, depth, 1) for i in range(num_units - 1)]


def get_blocks(num_layers):
    if num_layers == 18:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=2),
            get_block(in_channel=64, depth=128, num_units=2),
            get_block(in_channel=128, depth=256, num_units=2),
            get_block(in_channel=256, depth=512, num_units=2)
        ]
    elif num_layers == 34:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=6),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 50:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=14),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 100:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=13),
            get_block(in_channel=128, depth=256, num_units=30),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 152:
        blocks = [
            get_block(in_channel=64, depth=256, num_units=3),
            get_block(in_channel=256, depth=512, num_units=8),
            get_block(in_channel=512, depth=1024, num_units=36),
            get_block(in_channel=1024, depth=2048, num_units=3)
        ]
    elif num_layers == 200:
        blocks = [
            get_block(in_channel=64, depth=256, num_units=3),
            get_block(in_channel=256, depth=512, num_units=24),
            get_block(in_channel=512, depth=1024, num_units=36),
            get_block(in_channel=1024, depth=2048, num_units=3)
        ]

    return blocks


class Backbone(Module):
    def __init__(self, input_size, num_layers, mode='ir'):
        """ Args:
            input_size: input_size of backbone
            num_layers: num_layers of backbone
            mode: support ir or irse
        """
        super(Backbone, self).__init__()
        assert input_size[0] in [112, 224], \
            "input_size should be [112, 112] or [224, 224]"
        assert num_layers in [18, 34, 50, 100, 152, 200], \
            "num_layers should be 18, 34, 50, 100 or 152"
        assert mode in ['ir', 'ir_se'], \
            "mode should be ir or ir_se"
        self.input_layer = Sequential(Conv2d(3, 64, (3, 3), 1, 1, bias=False),
                                      BatchNorm2d(64), PReLU(64))
        blocks = get_blocks(num_layers)
        if num_layers <= 100:
            if mode == 'ir':
                unit_module = BasicBlockIR
            elif mode == 'ir_se':
                unit_module = BasicBlockIRSE
            output_channel = 512
        else:
            if mode == 'ir':
                unit_module = BottleneckIR
            elif mode == 'ir_se':
                unit_module = BottleneckIRSE
            output_channel = 2048

        if input_size[0] == 112:
            self.output_layer = Sequential(BatchNorm2d(output_channel),
                                        Dropout(0.4), Flatten(),
                                        Linear(output_channel * 7 * 7, 512),
                                        BatchNorm1d(512, affine=False))
        else:
            self.output_layer = Sequential(
                BatchNorm2d(output_channel), Dropout(0.4), Flatten(),
                Linear(output_channel * 14 * 14, 512),
                BatchNorm1d(512, affine=False))

        modules = []
        for block in blocks:
            for bottleneck in block:
                modules.append(
                    unit_module(bottleneck.in_channel, bottleneck.depth,
                                bottleneck.stride))
        self.body = Sequential(*modules)

        initialize_weights(self.modules())


    def forward(self, x):
        
        # current code only supports one extra image
        # it comes with a extra dimension for number of extra image. We will just squeeze it out for now
        x = self.input_layer(x)

        for idx, module in enumerate(self.body):
            x = module(x)

        x = self.output_layer(x)
        norm = torch.norm(x, 2, 1, True)
        output = torch.div(x, norm)

        return output, norm



def IR_18(input_size):
    """ Constructs a ir-18 model.
    """
    model = Backbone(input_size, 18, 'ir')

    return model


def IR_34(input_size):
    """ Constructs a ir-34 model.
    """
    model = Backbone(input_size, 34, 'ir')

    return model


def IR_50(input_size):
    """ Constructs a ir-50 model.
    """
    model = Backbone(input_size, 50, 'ir')

    return model


def IR_101(input_size):
    """ Constructs a ir-101 model.
    """
    model = Backbone(input_size, 100, 'ir')

    return model


def IR_152(input_size):
    """ Constructs a ir-152 model.
    """
    model = Backbone(input_size, 152, 'ir')

    return model


def IR_200(input_size):
    """ Constructs a ir-200 model.
    """
    model = Backbone(input_size, 200, 'ir')

    return model


def IR_SE_50(input_size):
    """ Constructs a ir_se-50 model.
    """
    model = Backbone(input_size, 50, 'ir_se')

    return model


def IR_SE_101(input_size):
    """ Constructs a ir_se-101 model.
    """
    model = Backbone(input_size, 100, 'ir_se')

    return model


def IR_SE_152(input_size):
    """ Constructs a ir_se-152 model.
    """
    model = Backbone(input_size, 152, 'ir_se')

    return model


def IR_SE_200(input_size):
    """ Constructs a ir_se-200 model.
    """
    model = Backbone(input_size, 200, 'ir_se')

    return model


adaface_models = {
    'ir_18':"/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/adaface_ir18_webface4m.ckpt",
}


def load_pretrained_model(architecture='ir_18'):
    # load model and pretrained statedict
    assert architecture in adaface_models.keys()
    model = build_model(architecture)
    statedict = torch.load(adaface_models[architecture])['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img[:,:,::-1] / 255.) - 0.5) / 0.5
    tensor = torch.tensor([brg_img.transpose(2,0,1)]).float()
    return tensor

In [11]:
adaFaceModel = load_pretrained_model('ir_18')

# print(images.shape)
# bgr_image = images[:, [2, 1, 0], :, :]
# print(bgr_image.shape)


#feature, _ = adaFaceModel(bgr_image)

In [12]:
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=FutureWarning) 
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print('Running on device: {}'.format(device))

Running on device: cuda:1


In [13]:
trainSet.data['age'].min()

-9

In [14]:
def get_normalized_age_value(original_age_value):
    return (original_age_value - trainSet.data['age'].min())/(trainSet.data['age'].max() - trainSet.data['age'].min())


def get_original_age_value(normalized_age_value):
    return normalized_age_value * (trainSet.data['age'].max()  - trainSet.data['age'].min()) + trainSet.data['age'].min()

In [15]:
from torch.nn.modules import MSELoss
from torch.nn.modules.loss import CrossEntropyLoss
from torch.nn.parallel import DataParallel


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(512, 256),
            # nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            # nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

class CompressionModel1(nn.Module):
    def __init__(self):
        super(CompressionModel1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return encoded

class CompressionModel2(nn.Module):
    def __init__(self):
        super(CompressionModel2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return encoded

class CombinedModel(nn.Module):
    def __init__(self, model1, model2):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        x = self.model1(x)
        x = self.model2(x)
        return x

class OgId(nn.Module):
    def __init__(self):
        super(OgId, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,80)
        )
    
    def forward(self, x):
        x = self.linear(x)
        return x

class CompId(nn.Module):
    def __init__(self):
        super(CompId, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,80)
        )
    
    def forward(self, x):
        x = self.linear(x)
        return x



class CompressionLoss(nn.Module):
    def __init__(self, batch_size):
        super(CompressionLoss, self).__init__()
        self.batch_size = batch_size

    def forward(self, img, genuine_imgs, imposter_imgs, genuine_sim, imposter_sim):
        # img -> base image embedding
        # genuine_imgs, imposter_imgs -> Compressed embeddings of dimension 128
        # genuine_sim, imposter_sim -> cosine smilairty scores of uncompressed embeddings of dimension 512


        # Compute cosine similarity loss
        genuine_loss = 0
        for i in range(len(genuine_imgs)):
            sim_score = F.cosine_similarity(img, genuine_imgs[i])
            genuine_loss += torch.mean(torch.abs(sim_score - genuine_sim[i].view(-1)))

        imp_loss = 0
        for i in range(len(imposter_imgs)):
            imp_sim_score = F.cosine_similarity(img, imposter_imgs[i])
            imp_loss += torch.mean(torch.abs(imp_sim_score - imposter_sim[i].view(-1)))

        
        # Compute Covariance Loss
        # matrix -> Concatenates compressed genuine and imposter embeddings
        
        matrix = torch.cat([genuine_imgs[0], imposter_imgs[0]], dim=0)
        for i in range(1, len(genuine_imgs)):
            matrix = torch.cat([matrix, genuine_imgs[i]], dim = 0)
            matrix = torch.cat([matrix, imposter_imgs[i]], dim = 0)


        mean_matrix = torch.mean(matrix, dim = 0)
        mx = torch.matmul(mean_matrix.t(), mean_matrix)
        vx = torch.matmul(matrix.t(), matrix) / self.batch_size # Dividing by batch size as done here https://github.com/human-analysis/hers-encrypted-image-search/blob/master/deep_mds%2B%2B/nntools/tensorflow/networks/deepmds.py#L94
        cov_matrix = mx - vx
        diag = torch.diag(cov_matrix.diag())
        cov_loss = torch.mean(torch.abs(cov_matrix - diag))
        
        # Compute Supervise Loss - Not used for ArcFace
        # # g_dist -> Concatenates compressed genuine embeddings
        # g_dist = torch.cat([genuine_imgs[0], genuine_imgs[1]], dim = 0)
        # for i in range(2, len(genuine_imgs)):
        #     g_dist = torch.cat([g_dist, genuine_imgs[i]], dim = 0)

        # # g_dist -> Concatenates compressed imposter embeddings
        # imp_dist = torch.cat([imposter_imgs[0], imposter_imgs[1]], dim = 0)
        # for i in range(2, len(imposter_imgs)):
        #     imp_dist = torch.cat([imp_dist, imposter_imgs[i]], dim = 0)

        # gloss = (torch.mean(g_dist) + 1.0) * 0.5
        # iloss = (torch.mean(imp_dist) + 1.0) * 0.5

        # lda_term = iloss / gloss
        # return genuine_loss + imp_loss + 10 * cov_loss 
        return 10*genuine_loss + imp_loss + cov_loss


In [29]:

class faceAnalytics(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.layer1=nn.Linear(128,64)
        self.dropout1=nn.Dropout(0.2)
        self.layer2=nn.Linear(64,32)
        #self.layer3=nn.Linear(1024,512)
        # self.layer4=nn.Linear(128,64)
        self.dropout2=nn.Dropout(0.2)
        self.genderOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        self.ageOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,4)
        )
        self.ethnicityOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        # self.identity = nn.Sequential(nn.Linear(32,64), nn.ReLU(), nn.Linear(64,80))
        # self.maxVal = 0
        # self.min=0
        
    
    def writeResult(self,result):
       output_directory=""
       file_name = "resultAge.txt"

       with open(os.path.join(output_directory, file_name), "w") as file:
        for value in result:
            file.write(f"{value}\n")
    
    def forward(self,x):

        #print("Input",x[0])
        # x=self.layer1(x)
        # x = nn.ReLU()(x)
        # x=self.layer2(x)
        # x = nn.ReLU()(x)

        gender=self.genderOut(x)
        age=self.ageOut(x)
        ethn = self.ethnicityOut(x)
        # identity = self.identity(x)
        return gender,age,ethn
    
    
    def are_values_in_same_class(self,value1, value2):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        for class_range in class_ranges:
            if class_range[0] <= value1 < class_range[1] and class_range[0] <= value2 < class_range[1]:
                return True
    
        return False    
    
    def trainAE(self, trainloader, testloader, adaFace, aeModel,OgIdLoss, CompIdLoss, device, episodes, completed_eps=0):
        input_dim = 512
        compressed_dim = 128
        model = aeModel
        # Define optimizer and loss function
        
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        MSELoss = nn.MSELoss().to(device)
        celoss = nn.CrossEntropyLoss().to(device)
        compLoss = CompIdLoss
        ogLoss = OgIdLoss  
        model.train()
        for ep in range(episodes):
            total_loss = 0
            print("Starting " + str(ep))
            cnt = 0

            for img, data in trainloader:
                cnt += 1
                if cnt%50 == 0:
                    print("Image number = " + str(cnt))
                img = img.to(device=device)
                label = data['name'].to(device=device)
                input = img[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings,_ = adaFace(input)
                encoded, decoded = model(embeddings)
                x1 = compLoss(encoded) 
                x2 = ogLoss(decoded)
                loss = celoss(x1, label) + celoss(x2, label) + 0.5*MSELoss(embeddings, decoded)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            if (ep+1) % 5 == 0:
                torch.save(model, f'AeModel_{ep+1+completed_eps}.pt')
            print(f"Epoch [{ep+completed_eps + 1}/{episodes+completed_eps}], Loss: {total_loss}")
        torch.save(model, f'AeModel_{episodes+completed_eps}.pt')
        print("Training complete!")


    def trainCombinedModel(self, trainloader, testloader, adaFace, combinedModel, compressionLoss, device, episodes, completed_eps=0):
        input_dim = 512
        compressed_dim = 128
        model = combinedModel
        # Define optimizer and loss function
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=4e-5)
        criterion = compressionLoss
        model.train()
        for ep in range(episodes):
            total_loss = 0
            print("Starting " + str(ep))
            cnt = 0
            for img, gen_pairs, imp_pairs in trainloader:

                # print(img.shape)
                # print(image_pairs[0].shape)
                # print(img)
                # print(len(image_pairs))
                
                cnt += 1
                if cnt%50 == 0:
                    print("Image number = " + str(cnt))
                

                img = img.to(device=device)
                input = img[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings,_ = adaFace(input)
                compressed_img = model(embeddings)
                gen_sim_scores = []
                gen_embedding_pairs = []
                for img_pair in gen_pairs:

                    img_pair = img_pair.to(device=device)
                    input_pair = img_pair[:, [2, 1, 0], :, :]
                    #print(inputs.shape)
                    embedding_pair,_ = adaFace(input_pair)
                    compressed_img2 = model(embedding_pair)
                    gen_embedding_pairs.append(compressed_img2)
                    sim_score = F.cosine_similarity(embeddings, embedding_pair)
                    gen_sim_scores.append(sim_score)
                
                imp_sim_scores = []
                imp_embedding_pairs = []
                for img_pair in imp_pairs:

                    img_pair = img_pair.to(device=device)
                    input_pair = img_pair[:, [2, 1, 0], :, :]
                    #print(inputs.shape)
                    embedding_pair,_ = adaFace(input_pair)
                    compressed_img2 = model(embedding_pair)
                    imp_embedding_pairs.append(compressed_img2)
                    sim_score = F.cosine_similarity(embeddings, embedding_pair)
                    imp_sim_scores.append(sim_score)
                
                loss = criterion(compressed_img, gen_embedding_pairs, imp_embedding_pairs, gen_sim_scores, imp_sim_scores)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            if (ep+1) % 5 == 0:
                torch.save(model, f'CombinedModel_{ep+1+completed_eps}.pt')
            print(f"Epoch [{ep+completed_eps + 1}/{episodes+completed_eps}], Loss: {total_loss}")
        torch.save(model, f'CombinedModel_{episodes+completed_eps}.pt')
        print("Training complete!")

    def trainCompressionModel2(self, trainloader, testloader, adaFace, compressionModel1, compressionModel2, compressionLoss, device, episodes, completed_eps = 0):
        
        # Define optimizer and loss function
        compressionModel1.eval()
        optimizer = optim.Adam(compressionModel2.parameters(), lr=0.001, weight_decay=4e-5)
        criterion = compressionLoss
        compressionModel2.train()
        for ep in range(episodes):
            total_loss = 0
            print("Starting " + str(ep))
            cnt = 0
            for img, gen_pairs, imp_pairs in trainloader:

                # print(img.shape)
                # print(image_pairs[0].shape)
                # print(img)
                # print(len(image_pairs))
                
                cnt += 1
                if cnt%50 == 0:
                    print("Image number = " + str(cnt))
                

                img = img.to(device=device)
                input = img[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings,_ = adaFace(input)
                compressed_img = compressionModel1(embeddings)
                compressed_img2 = compressionModel2(compressed_img)
                gen_sim_scores = []
                gen_embedding_pairs = []
                for img_pair in gen_pairs:

                    img_pair = img_pair.to(device=device)
                    input_pair = img_pair[:, [2, 1, 0], :, :]
                    #print(inputs.shape)
                    embedding_pair,_ = adaFace(input_pair)
                    compression_pair1 = compressionModel1(embedding_pair)
                    compressed_pair2 = compressionModel2(compression_pair1)
                    gen_embedding_pairs.append(compressed_pair2)
                    sim_score = F.cosine_similarity(embeddings, embedding_pair)
                    gen_sim_scores.append(sim_score)
                
                imp_sim_scores = []
                imp_embedding_pairs = []
                for img_pair in imp_pairs:

                    img_pair = img_pair.to(device=device)
                    input_pair = img_pair[:, [2, 1, 0], :, :]
                    #print(inputs.shape)
                    embedding_pair,_ = adaFace(input_pair)
                    compression_pair1 = compressionModel1(embedding_pair)
                    compressed_pair2 = compressionModel2(compression_pair1)
                    imp_embedding_pairs.append(compressed_pair2)
                    sim_score = F.cosine_similarity(embeddings, embedding_pair)
                    imp_sim_scores.append(sim_score)
                
                loss = criterion(compressed_img2, gen_embedding_pairs, imp_embedding_pairs, gen_sim_scores, imp_sim_scores)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            if (ep + 1) % 5 == 0:
                torch.save(compressionModel2, f'CompressionModel2_{ep+1+completed_eps}.pt')
            print(f"Epoch [{completed_eps + ep + 1}/{completed_eps + episodes}], Loss: {total_loss}")
        torch.save(compressionModel2, f'CompressionModel2_{completed_eps + episodes}.pt')
        print("Training complete!")

    def trainCompressionModel1(self, trainloader, testloader, adaFace, compressionModel1, compressionLoss, device, episodes, completed_eps = 0):
        input_dim = 512
        compressed_dim = 128
        model = compressionModel1
        # Define optimizer and loss function
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        criterion = compressionLoss
        model.train()
        for ep in range(episodes):
            total_loss = 0
            print("Starting " + str(ep))
            cnt = 0
            for img, gen_pairs, imp_pairs in trainloader:

                # print(img.shape)
                # print(image_pairs[0].shape)
                # print(img)
                # print(len(image_pairs))
                
                cnt += 1
                if cnt%50 == 0:
                    print("Image number = " + str(cnt))
                

                img = img.to(device=device)
                input = img[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings,_ = adaFace(input)
                compressed_img = model(embeddings)
                gen_sim_scores = []
                gen_embedding_pairs = []
                for img_pair in gen_pairs:

                    img_pair = img_pair.to(device=device)
                    input_pair = img_pair[:, [2, 1, 0], :, :]
                    #print(inputs.shape)
                    embedding_pair,_ = adaFace(input_pair)
                    compressed_img2 = model(embedding_pair)
                    gen_embedding_pairs.append(compressed_img2)
                    sim_score = F.cosine_similarity(embeddings, embedding_pair)
                    gen_sim_scores.append(sim_score)
                
                imp_sim_scores = []
                imp_embedding_pairs = []
                for img_pair in imp_pairs:

                    img_pair = img_pair.to(device=device)
                    input_pair = img_pair[:, [2, 1, 0], :, :]
                    #print(inputs.shape)
                    embedding_pair,_ = adaFace(input_pair)
                    compressed_img2 = model(embedding_pair)
                    imp_embedding_pairs.append(compressed_img2)
                    sim_score = F.cosine_similarity(embeddings, embedding_pair)
                    imp_sim_scores.append(sim_score)
                
                loss = criterion(compressed_img, gen_embedding_pairs, imp_embedding_pairs, gen_sim_scores, imp_sim_scores)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            if (ep+1) % 5 == 0:
                torch.save(model, f'CompressionModel1_{ep+1+completed_eps}.pt')
            print(f"Epoch [{ep + 1 + completed_eps}/{completed_eps + episodes}], Loss: {total_loss}")
        torch.save(model, f'CompressionModel1_{episodes+completed_eps}.pt')
        print("Training complete!")


    def trainModel(self,trainLoader,testLoader,adaFace, compressionModel, device,episodes):
        self.train()
        #maxVal = 0
        learningRate=0.01
        gender_loss = nn.CrossEntropyLoss() 
        age_loss = nn.CrossEntropyLoss() 
        ethn_loss = nn.CrossEntropyLoss()
        identityLoss = nn.CrossEntropyLoss()
        #optimizer = torch.optim.Adam(self.parameters(), lr=learningRate)
        optimizer = torch.optim.SGD(self.parameters(), lr=learningRate,
        momentum=0.9, weight_decay=5e-5)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=episodes)
        trainingAcc = []
        model = compressionModel
        for e in range(0,episodes):
            total_training_loss =0
            genderAcc=0
            ethnAcc = 0
            ageAcc=0
            identAcc=0
            count=0
            totalGenderLoss=0


            for i,data in enumerate(trainLoader):

                #print(inputs.shape)
                #inputs = inputs.type(torch.double)
                
                #gender_label = data["gender"].to(device=device)
                #race_label = data["race"].to(device=device)
            # print(i)
                inputs = data[0].to(device=device)
                #inputs = inputs.type(torch.double)
                age_label = (data[1]["age"]).to(device=device)
            
                gender_label = data[1]["gender"].to(device=device)
                ethn_label = data[1]["ethnicity"].to(device=device)

                ident_label = data[1]["name"].to(device=device)
                
                #print(torch.max(ethn_label),torch.min(ethn_label))
                #print(torch.max(gender_label),torch.min(gender_label))

                #print(count)
                #print(inputs.shape)
                inputs = inputs[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings,_ = adaFace(inputs)
                encoded, _ = model(embeddings)
                gender,age,ethn = self(encoded)
                ageLoss = age_loss(age,age_label)
                loss = gender_loss(gender,gender_label) + ageLoss + ethn_loss(ethn,ethn_label)
                # loss = gender_loss(gender,gender_label) + ageLoss + ethn_loss(ethn,ethn_label)  + identityLoss(ident,ident_label)
                #print(gender)
                #print(gender_label)
                #totalGenderLoss = totalGenderLoss + loss.item()
                loss.backward()
                #print("Loss:",loss)
                optimizer.step()
                optimizer.zero_grad()
                #total_training_loss = total_training_loss+loss.item()*512

                predictedGender = torch.argmax(gender,dim=1)
                predictedEthn = torch.argmax(ethn,dim=1)
                predictedAge = torch.argmax(age,dim=1)
                # predictedIdent = torch.argmax(ident,dim=1)
                #print(predictedGender)
                #print(gender_label)
                for j in range(0,predictedGender.shape[0]):
                    count=count+1
                    #print(predictedGender[j].item(),gender_label[j])
                    if(predictedGender[j].item()==gender_label[j].item()):
                        genderAcc=genderAcc+1
                
                    if(predictedEthn[j].item()==ethn_label[j].item()):
                        ethnAcc=ethnAcc+1
    
                    if(predictedAge[j].item()==age_label[j].item()):
                        ageAcc=ageAcc+1

                    # if(predictedIdent[j].item()==ident_label[j].item()):
                    #     identAcc=identAcc+1
    
            genderAccuracy =  genderAcc/count
            trainingAcc.append(genderAccuracy)
            print("Gender Accuracy:", genderAccuracy,"Age Acc:", ageAcc/count, " ethnAcc:", ethnAcc/count)
            #print("total training loss:",total_training_loss/16595,"\n")
            #print("\n")
            scheduler.step()
            #print("max observed value: ", maxVal)
            if(e%2==0):
                self.test(testLoader,adaFace, compressionModel, device)
            
        return trainingAcc

    def are_values_in_same_class(self,value1, value2):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        for class_range in class_ranges:
            if class_range[0] <= value1 < class_range[1] and class_range[0] <= value2 < class_range[1]:
                return True
    
        return False

    def test(self,testLoader,adaFace, compressionModel, device):

         self.eval()
         age_loss = nn.L1Loss()

         totalAgeError = 0
         genderAccuracy = 0
         tempAcc=0
         count=0
         totalGenderLoss=0
         maxAge = 0
         minAge = 0
         ageAccuracy = 0
         total_training_loss =0
         tempAcc=0
         count=0
         totalGenderLoss=0

         genderAcc=0
         ethnAcc = 0
         identAcc = 0

         model = compressionModel
        
         for i,data in enumerate(testLoader):

            #print(inputs.shape)
            #inputs = inputs.type(torch.double)
            
            #gender_label = data["gender"].to(device=device)
            #race_label = data["race"].to(device=device)
            
   
            #inputs = inputs.type(torch.double)
            inputs = data[0].to(device=device)
            age_label = data[1]["age"].to(device=device)
            gender_label = data[1]["gender"].to(device=device)
            ethn_label = data[1]["ethnicity"].to(device=device)
            ident_label = data[1]['name'].to(device=device)
            #print(count)
            inputs = inputs[:, [2, 1, 0], :, :]
            embeddings,_ = adaFace(inputs)
            compressedEm, _ = model(embeddings)
            gender,age,ethn = self(compressedEm)
         

            predictedGender = torch.argmax(gender,dim=1)
            predictedEthn = torch.argmax(ethn,dim=1)
            predictedAge = torch.argmax(age,dim=1)
            # predictedIdent = torch.argmax(ident,dim=1)
            #age = get_original_age_value(age)

            for j in range(0,predictedGender.shape[0]):
                count=count+1
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedGender[j].item()==gender_label[j]):
                    genderAcc=genderAcc+1
                if(predictedEthn[j].item()==ethn_label[j]):
                    ethnAcc=ethnAcc+1
                if(predictedAge[j].item()==age_label[j]):
                   ageAccuracy = ageAccuracy +1
                
                # if(predictedIdent[j].item()==ident_label[j].item()):
                #     identAcc=identAcc+1

         genderAccuracy =  genderAcc/count
         
         print("Test Gender Accuracy:", genderAccuracy,"Test Age Accuracy:", ageAccuracy/count, "Test ethnicity Accuracy:", ethnAcc/count)

         return genderAccuracy,totalAgeError

In [17]:
torch.cuda.device_count()

4

In [18]:
batch_size = 8
FAmodel=faceAnalytics()
# compressionModel1 = CompressionModel1()
# compressionModel2 = CompressionModel2()
# compressionLoss = CompressionLoss(batch_size)
# combinedModel = CombinedModel(compressionModel1, compressionModel2)
aeModel = Autoencoder()
#model=torch.load("arcFaceCelebSetBase.pt")
FAmodel.to(device)
adaFaceModel.to(device)
adaFaceModel.eval()
# compressionModel1.to(device)
# compressionModel2.to(device)
# combinedModel.to(device)
aeModel.to(device)
# compressionLoss = DataParallel(compressionLoss)
# compressionLoss.to(device)
#torch.save(model,"bestFaceAn.pt")

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): Sigmoid()
  )
)

In [19]:
ogLoss = OgId()
compLoss = CompId()
ogLoss.to(device)
compLoss.to(device)
FAmodel.trainAE(trainloader, testloader, adaFaceModel, aeModel, ogLoss, compLoss, device, 20, 0)

Starting 0
Image number = 50
Image number = 100
Image number = 150
Image number = 200
Image number = 250
Image number = 300
Image number = 350
Image number = 400
Image number = 450
Image number = 500
Image number = 550
Image number = 600
Image number = 650
Image number = 700
Image number = 750
Image number = 800
Image number = 850
Image number = 900
Image number = 950
Image number = 1000
Image number = 1050
Image number = 1100
Image number = 1150
Image number = 1200
Image number = 1250
Image number = 1300
Image number = 1350
Image number = 1400
Image number = 1450
Image number = 1500
Image number = 1550
Image number = 1600
Image number = 1650
Image number = 1700
Image number = 1750
Image number = 1800
Image number = 1850
Image number = 1900
Image number = 1950
Image number = 2000
Image number = 2050
Image number = 2100
Image number = 2150
Image number = 2200
Epoch [1/20], Loss: 18630.683947086334
Starting 1
Image number = 50
Image number = 100
Image number = 150
Image number = 200
Imag

In [ ]:

# FAmodel=faceAnalytics()
# FAmodel.to(device)
FAmodel.trainCompressionModel1(trainloader,testloader,adaFaceModel,compressionModel1, compressionLoss, device, 20, 10)

In [ ]:

# FAmodel=faceAnalytics()
# FAmodel.to(device)
FAmodel.trainCompressionModel2(trainloader,testloader,adaFaceModel,compressionModel1, compressionModel2, compressionLoss, device, 10,50)

In [ ]:

FAmodel=faceAnalytics()
FAmodel.to(device)
compressionModel1.eval()
compressionModel2.eval()
FAmodel.trainCombinedModel(trainloader, testloader, adaFaceModel, combinedModel, compressionLoss, device, 10, 10)

In [20]:
warnings.filterwarnings("ignore", category=UserWarning)


processedTensor = torch.zeros((len(testSet), 128))
identityLabel = torch.zeros((len(testSet)))
count = 0

for i,data in enumerate(testloader):
    
    inputs = data[0].to(device=device)
    iden_label = data[1]['name'].to(device=device)
    inputs = inputs[:, [2, 1, 0], :, :]    
    embeddings,_ = adaFaceModel(inputs)
    encoded, _ = aeModel(embeddings)
    # compressed_img1 = compressionModel1(embeddings)
    # compressed_img2 = compressionModel2(compressed_img1)
    # compressed = combinedModel(embeddings)
    processedTensor[count : count + embeddings.shape[0]] = torch.tensor(encoded)
    identityLabel[count:count + iden_label.shape[0]] = torch.tensor(iden_label)


    count = count + embeddings.shape[0]
    

In [21]:
identityAccuracy = 0

count = 0

while(count < processedTensor.shape[0]):

    currentFace = processedTensor[count : count + 1]
    cosine_similarity = nn.functional.cosine_similarity(currentFace, processedTensor)
    if(identityLabel[torch.topk(cosine_similarity, k = 2)[1][1]].item() == identityLabel[count].item()):
        identityAccuracy = identityAccuracy +1
    count = count + 1
    # print(count)
print(identityAccuracy/count)

0.8325


In [ ]:
# FAmodel.trainCompressionModel(trainloader,testloader,adaFaceModel,compressionModel, compressionLoss, device, 10)

In [ ]:
trainSetDet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv', get_pairs=False, transform=transform)
trainloaderDet = DataLoader(trainSetDet, batch_size=16, shuffle=True, num_workers=2)

testSetDet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv', get_pairs=False, transform=transform)
testloaderDet = DataLoader(testSetDet, batch_size=16, shuffle=False, num_workers=2)


In [31]:
aeModel = torch.load('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/CompressionCode/AeModel_20_bs32_82acc_best.pt')
aeModel.to(device)
FAmodel=faceAnalytics()
FAmodel.to(device)
aeModel.eval()
trainingAcc = FAmodel.trainModel(trainloader,testloader,adaFaceModel,aeModel, device,5)

Gender Accuracy: 0.9623919084766325 Age Acc: 0.6096150319513606  ethnAcc: 0.9649875156935491
Test Gender Accuracy: 0.934375 Test Age Accuracy: 0.571875 Test ethnicity Accuracy: 0.925625
Gender Accuracy: 0.969106631494308 Age Acc: 0.5989786849863872  ethnAcc: 0.9704608613466124
Gender Accuracy: 0.971490640296802 Age Acc: 0.5989786849863872  ethnAcc: 0.97325396041699
Test Gender Accuracy: 0.954375 Test Age Accuracy: 0.571875 Test ethnicity Accuracy: 0.921875
Gender Accuracy: 0.9732398538560284 Age Acc: 0.5989786849863872  ethnAcc: 0.9745235509035253


KeyboardInterrupt: 

In [ ]:
FAmodel.test(testloaderDet,adaFaceModel, compressionModel, device)

In [ ]:
torch.save(model,"arcFaceCelebSetBase.pt")

In [ ]:
torch.save(model,"negGenderBaseWiki.pt")

In [ ]:
plt.plot(trainingAcc)

In [ ]:
torch.save(model,"model92.pt")

In [ ]:
count=0
input1=[]
for i,data in enumerate(val_dataloader):
    
    if(count==0):
     inputs=resnet(data["image"].to(device))

 
     input1 = polyprotect(0,inputs[0])

     break
    count=count+1

In [ ]:
input1

In [ ]:
for param in model.parameters():
    print(param.shape)
    print(param[0])
    print(torch.dot(input1,param[1]))
    break 

In [ ]:
input1

In [ ]:
output_directory=""
file_name = "input1.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in input1:
            file.write(f"{value}\n")

In [ ]:
model=faceAnalytics()
model=torch.load("/home/csgrad/byalavar/FHE/HEAAN/modelUsing0.pt")
model.to(device)
model.test(dataloader,device)

In [ ]:
torch.save(model,"modelUsing0.pt")

In [ ]:
count=0
for param in model.parameters():
    print(param.shape)
    count=count+1
    if(count==2):
        print(param)

In [ ]:
a=nn.Linear(4,2)
input1=torch.rand((1,4))
print("input1",input1)
for param in a.parameters():
    print("param",param)
print(a(input1))


In [ ]:
model=faceAnalytics()
model=torch.load("modelUsing0.pt")
model.to(device)
count=0
ageBias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==7):
       ageBias = param.tolist()
    count=count+1
print(len(ageBias),len(ageBias[0]))


In [ ]:
ageBias

In [ ]:
bias2

In [ ]:
# Create a list of lists with shape (512, 128)


# Define the directory where you want to save the text files
output_directory = "ageWeights"

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

# Write each list to a separate text file


# Write each list to a separate text file
for i, sublist in enumerate(ageWeights):
    # Define the file name with leading zeros
    file_name = f"{i:03d}.txt"

    # Write each value in the sublist on a new line
    with open(os.path.join(output_directory, file_name), "w") as file:
        for value in sublist:
            file.write(f"{value}\n")

print("Files saved successfully.")


In [ ]:
count=1
layer1Bias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==2):
       layer1Bias = param.tolist()
       break
    count=count+1
print(len(layer1Bias),len(layer1Bias[0]))

In [ ]:
output_directory=""
file_name = "ageBias.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in ageBias:
            file.write(f"{value}\n")